<a href="https://colab.research.google.com/github/aayush2710/CTW2020-IDP/blob/master/TRRS_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls "/content/gdrive/My Drive/CTW_2019"

In [ ]:
import numpy as np # linear algebra
import keras
import os
import h5py
import pandas as pd
import time
import matplotlib.pyplot as plt
import tensorflow as tf

Using TensorFlow backend.


In [ ]:
'''#getting and processing data
#h_Estimated   ---> shape = (2, 924, 16, 17486)
#r_postition ---> shape = (3, 17486)
#SNR_Est    ---> shape = (16, 17486)


batch_size = 30
epochs = 50
data_augmentation = False
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cnn_training.h5'
'''

In [ ]:
#data fetching

path = "/content/gdrive/My Drive/CTW2020/CTW2020_labelled_data/file_" + str(1) +".hdf5"
f = h5py.File(path, 'r')
obj = enumerate(f.keys())
_, key  = next(obj)
H_RE  = f[key][:]
_, key  = next(obj)
H_IM  = f[key][:]
_, key  = next(obj)
POS  = f[key][:]
f.close()

H = np.concatenate((H_RE[:,:,:,:,np.newaxis], H_IM[:,:,:,:,np.newaxis]), axis=4)
del H_RE, H_IM, obj, key , _



for i in range(2,10):
    path = "/content/gdrive/My Drive/CTW2020/CTW2020_labelled_data/file_" + str(i) +".hdf5"
    f = h5py.File(path, 'r')
    obj = enumerate(f.keys())
    _, key  = next(obj)
    H_RE  = f[key][:]
    _, key  = next(obj)
    H_IM  = f[key][:]
    _, key  = next(obj)
    POSt  = f[key][:]
    f.close()
    Ht = np.concatenate((H_RE[:,:,:,:,np.newaxis], H_IM[:,:,:,:,np.newaxis]), axis=4)
    H = np.concatenate((H, Ht), axis = 0)
    POS = np.concatenate((POS, POSt), axis = 0)
    print(POS.shape, H.shape)




In [ ]:
H = np.swapaxes(H, 1,2)

In [ ]:
H.shape

In [ ]:
idx  = POS[POS[:,0] < 70]
idx = np.array(idx)
idx.shape

In [ ]:
H = H[POS[:,0] < 70]
POS = POS[POS[:,0] < 70]
del idx
print(H.shape, POS.shape)

In [ ]:
data = H[:,:,:,:1000]
print(data.shape)
xyz = pos[:,:1000]
print(xyz.shape)

In [ ]:
#Reducing the area , since not enough memory for the whole data.


H = data[:,:,:,:900]
H = H.swapaxes(0, -1)
pos = xyz[:,:900]

H_test = data[:,:,:,900:]
H_test = H_test.swapaxes(0, -1)
pos_test = xyz[:,900:]

print(H.shape)
print(pos_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split

H, H_test, POS, POS_t = train_test_split(H, POS, random_state = 42, test_size = 0.1)
print(H.shape, H_test.shape, POS.shape, POS_t.shape)


In [ ]:
X_train = np.swapaxes(X_train, 0,3)
H11 = H1[:,:,0] + H1[:,:,1] * 1j
X_test = np.swapaxes(X_test, 0,3)
print(X_train.shape, X_test.shape)

In [ ]:
#TRRS algo

def trrs(H1, H2):
    Yd1 = np.sum((H1*H1)[:,:,:,:,0] + (H1*H1)[:,:,:,:,1] , axis=1)
    Yd2 = np.sum((H2*H2)[:,:,:,0] + (H2*H2)[:,:,:,1] , axis=0)
    #print((Yd1*Yd2).shape) (n,56,5)*(56,5) == (n,56,5)

    H11 = H1[:,:,:,:,0] + H1[:,:,:,:,1] * 1j
    H22 = H2[:,:,:,0] - H2[:,:,:,1] * 1j
    G = H11 * H22
    #(n,924,56,5)
    
    Nser = 1024
    Nu = 924
    G = np.concatenate((G, np.zeros((H1.shape[0],Nser-Nu, 56,5))), axis=1)
    #print(G.shape)
    #(n,1024,56,5)
    
    g = np.fft.fftn(G, axes=(1,))
    #print(g.shape)  #(n,1024,56,5)


    phi_d = np.ndarray.max((np.square(np.abs(g))), axis=1)
    phi_d = np.divide(phi_d,(Yd1*Yd2))
    #print(phi_d.shape) #(n,56,5)

    w_d = np.sqrt((Yd1 * Yd2))
    w_d = np.swapaxes(w_d,0,1)/(np.sqrt(np.sum(Yd1, axis=1)) * np.sum(Yd2, axis = 0))
    w_d = np.swapaxes(w_d,0,1)
    #print(w_d.shape) #(n, 56, 5)
    
    TRRS = np.sum(np.multiply(w_d,np.sqrt(phi_d)), axis=1)
    TRRS = np.square(TRRS)
    TRRS = np.mean(TRRS, axis = 1)
    idx = np.argmax(TRRS)

    return POS[idx,:]


print(trrs(H, H_test[28]))
print(POS_t[48])


In [ ]:
#Same algo as above just easy to use interface
class TRRS:
    def __init__(self, CFR , CFR_Pos):
        self.CFR = CFR
        self.CFR_Pos = CFR_Pos
  
    def get_fingerprint_subset(self , pos ,radius = 10):
        out = []
        for i in range(len(self.CFR_Pos)):
            if np.sqrt(np.sum((pos-self.CFR_Pos[i])**2)) < radius:
                out.append(i)
        return np.array(out, dtype=int)

    def predict(self , H2, Pos2=[] , radius=10):
        if len(Pos2)==0:
            idx = range(len(self.CFR))
        else:
            idx = self.get_fingerprint_subset(Pos2,radius)
        H1 = self.CFR[idx]
        POS = self.CFR_Pos[idx]
        Yd1 = np.sum((H1*H1)[:,:,:,:,0] + (H1*H1)[:,:,:,:,1] , axis=1)
        Yd2 = np.sum((H2*H2)[:,:,:,0] + (H2*H2)[:,:,:,1] , axis=0)
        #print((Yd1*Yd2).shape) (n,56,5)*(56,5) == (n,56,5)

        H11 = H1[:,:,:,:,0] + H1[:,:,:,:,1] * 1j
        H22 = H2[:,:,:,0] - H2[:,:,:,1] * 1j
        G = H11 * H22
        #(n,924,56,5)
        
        Nser = 1024
        Nu = 924
        G = np.concatenate((G, np.zeros((H1.shape[0],Nser-Nu, 56,5))), axis=1)
        #print(G.shape)TRRS
        #(n,1024,56,5)
        
        g = np.fft.fftn(G, axes=(1,))
        #print(g.shape)  #(n,1024,56,5)


        phi_d = np.ndarray.max((np.square(np.abs(g))), axis=1)
        phi_d = np.divide(phi_d,(Yd1*Yd2))
        #print(phi_d.shape) #(n,56,5)

        w_d = np.sqrt((Yd1 * Yd2))
        w_d = np.swapaxes(w_d,0,1)/(np.sqrt(np.sum(Yd1, axis=1)) * np.sum(Yd2, axis = 0))
        w_d = np.swapaxes(w_d,0,1)
        #print(w_d.shape) #(n, 56, 5)
        
        TRRS = np.sum(np.multiply(w_d,np.sqrt(phi_d)), axis=1)
        TRRS = np.square(TRRS)
        TRRS = np.max(TRRS, axis = 1)
        try:
            idx = np.argmax(TRRS)
        except:
            return self.pos2

        
        return POS[idx,:]

In [ ]:
trrs = TRRS(H, POS)

In [ ]:
#Trying to simulate NN + TRRS by adding arbitary noise and then predicting using TRRS

test_sample = np.random.randint(0,POS_t.shape[0])
plt.scatter(POS[:,0], POS[:,1])
plt.scatter(POS_t[test_sample,0], POS_t[test_sample,1])
print(POS_t[test_sample])


def add_noise(pos):
    gg = np.zeros((pos.shape))
    gg[:2]  = pos[:2] + np.random.normal(4,2,2)
    gg[-1] = pos[-1] + np.random.normal(0,1,1)
    return gg


print(POS_t[test_sample], add_noise(POS_t[test_sample]))

In [ ]:
sample = np.array(trrs.predict(H_test[test_sample], add_noise(POS_t[test_sample]), radius=30))
print(sample)

print(np.sqrt(np.sum(np.square(POS_t[test_sample] -add_noise(POS_t[test_sample])))))
print(np.sqrt(np.sum(np.square(add_noise(POS_t[test_sample]) -sample))))
print(np.sqrt(np.sum(np.square(POS_t[test_sample] -sample))))

In [ ]:
samples = []
noisy = []


for i in range(POS_t.shape[0]):
    temp = add_noise(POS_t[i])
    noisy.append(temp)
    samples.append(trrs.predict(H_test[i], temp, radius=30))

    print("%d / %d" %(i+1, POS_t.shape[0]))




In [ ]:
samples, noisy = np.array(samples) , np.array(noisy)

print(np.mean(np.sqrt(np.sum(np.square(noisy - POS_t), axis = 1))))
print(np.mean(np.sqrt(np.sum(np.square(samples-POS_t), axis=1))))


In [ ]:
import time as time
ans_grid = np.zeros((POS_t.shape))

for i in range(POS_t.shape[0]):
    start_time = time.time()
    ans_grid[i] = trrs(H,H_test[i])
    print(ans_grid[i], POS_t[i])
    print("%d / %d"%(i, POS_t.shape[0]))
    print("<------Time elapsed %s ------>" %(time.time()-start_time))

err = np.sqrt(((ans_grid - pos_test) ** 2).mean())

print(err)


In [ ]:
#Testing

import time
start_time = time.time()
errors = []
for idx, ele in enumerate(X_test):
  sample_time = time.time()
  print(idx, ele.shape)
  errors.append(np.sqrt(np.sum((y_test[:,idx] - trrs(X_train, ele))**2)))
  print(errors[idx])
  print("------%s seconds for this sample ------->>>>>>>>>>>>>>>>>>>>>>>>>" % (time.time()-sample_time))
print(np.mean(errors))
print("--- %s seconds ---" % (time.time() - start_time))
